In [3]:
import pandas as pd
import pymysql
import tokenkey as tky

In [18]:
df = pd.read_csv('D:\Program File\Python\AllEnv\envlatihan\Latihan\RequestAPI\Youtube\youtube_pull.csv')
print(df.head())

      video_id                                        video_title upload_date  \
0  30hS-MjpU6E  SQL Syntax Best Practices: How to Structure Yo...  2021-12-22   
1  Bgpp99iz0I0  Most Common Coding Mistakes on Data Science In...  2021-11-04   
2  GeJUvdkJKEc  Multiple Solutions to Data Scientist Interview...  2021-10-11   
3  GGURenNfXI0  Multiple Approaches for Solving a Twitter Data...  2021-09-13   
4  PlpUo6bHsBQ  Advanced Facebook Data Science SQL interview q...  2021-08-09   

   view_count  like_count  dislike_count  comment_count  
0        2978         182            NaN             18  
1        3108         153            NaN             27  
2        4151         126            NaN             32  
3        3229         144            NaN             32  
4       10621         341            NaN             45  


In [6]:
db = pymysql.connect(host=tky.HOST_NAME_mysql, user=tky.USERNAME_mysql, password=tky.PASSWORD_mysql)
db

In [7]:
cursor = db.cursor()
cursor

# DDL

In [10]:
sql = '''create database IF NOT EXISTS test'''
cursor.execute(sql)

1

In [11]:
cursor.connection.commit()

In [12]:
sql = '''use test'''
cursor.execute(sql)

0

In [14]:
sql = """CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    dislike_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )"""
cursor.execute(sql)

0

In [15]:
sql = '''show tables'''
cursor.execute(sql)
cursor.fetchall()

(('videos',),)

# DML

In [24]:
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM videos WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['comment_count'])


# Buat liat misal datanya udah ada dia cuma bakal update doang datanya berdasarkan idnya, klo belom ada di append ke temp_df dulu
# Nanti temp_df bakal di append ke db lewat func append_from_df_to_db()
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [25]:
new_vid_df = update_db(cursor, df)

In [26]:
new_vid_df

,video_id,video_title,upload_date,view_count,like_count,comment_count,dislike_count
0,30hS-MjpU6E,SQL Syntax Best Practices: How to Structure Yo...,2021-12-22,2978,182,18,NaN
1,Bgpp99iz0I0,Most Common Coding Mistakes on Data Science In...,2021-11-04,3108,153,27,NaN
2,GeJUvdkJKEc,Multiple Solutions to Data Scientist Interview...,2021-10-11,4151,126,32,NaN
3,GGURenNfXI0,Multiple Approaches for Solving a Twitter Data...,2021-09-13,3229,144,32,NaN
4,PlpUo6bHsBQ,Advanced Facebook Data Science SQL interview q...,2021-08-09,10621,341,45,NaN
...,...,...,...,...,...,...,...
74,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,1726,12,2,NaN
75,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,956,13,0,NaN
76,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,898,17,0,NaN
77,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,7451,63,1,NaN


In [27]:
append_from_df_to_db(cursor, new_vid_df)

In [28]:
cursor.connection.commit()

In [31]:
cursor.execute("SELECT * FROM videos")
cursor.fetchall()

(('_gy1o9UH2dQ',
  'Hard SQL Interview Question From FACEBOOK | Data Science Coding Interviews (Popularity Percentage)',
  datetime.date(2020, 11, 4),
  16254,
  338,
  0,
  58),
 ('-i9chjBSOqU',
  'merge() Method to Join Multiple Pandas Dataframes - Beginner Python Pandas Tutorial #7',
  datetime.date(2020, 5, 29),
  928,
  20,
  0,
  5),
 ('01kOJUiiEQE',
  'NumPy Slicing and Accessing Subarrays - Complete Python NumPy Tutorial for Beginners #5',
  datetime.date(2020, 5, 14),
  267,
  2,
  0,
  0),
 ('0GpgMvyN0Fg',
  'How I Learned Data Science (resources to get a job) in 2021',
  datetime.date(2021, 2, 4),
  8379,
  547,
  0,
  44),
 ('0hjTJw9Jpmc',
  'How to Reshape NumPy Arrays - Beginner Python NumPy Exercises #5',
  datetime.date(2020, 5, 14),
  235,
  3,
  0,
  0),
 ('1SgHHQeRuxs',
  'How I Prepare For My Data Science Interviews (From a Data Scientist)',
  datetime.date(2020, 10, 18),
  4174,
  117,
  0,
  17),
 ('2iE3JgNTwVU',
  'A Walkthrough of Data Science SQL Interview Ques